<a href="https://colab.research.google.com/github/YCYoon2015/DL_zero_to_all_PyTorch/blob/master/210730_ch9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transform
import random

In [7]:
random.seed(777) # 시드 고정하여 재현성 확보
torch.manual_seed(777)

In [8]:
learning_rate = 0.001 # 너무크면 발산하고, 너무 작으면 효율적이지 못하다
training_epochs = 15
batch_size = 100

In [9]:
# minist 데이터를 불러온다. 인자 설명은 공식 문서에 있다.
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transform.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transform.ToTensor(),
                         download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [12]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [15]:
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[ 1.5347,  1.0241,  0.0551,  ...,  0.2334, -1.0393, -0.1661],
        [-0.7286, -0.6023,  0.7702,  ..., -0.3004,  0.2032,  0.9696],
        [ 0.8494,  0.8011,  0.3364,  ...,  0.5827, -3.0999, -0.5124],
        ...,
        [-1.3700, -1.1506,  0.0155,  ..., -0.8441, -0.7930,  0.0077],
        [-1.0760,  0.1694,  0.9790,  ..., -1.3515,  0.1347, -0.6594],
        [ 1.3775, -0.6955,  0.6767,  ...,  0.9700, -2.5318,  0.5630]],
       requires_grad=True)

In [18]:
criterion = torch.nn.CrossEntropyLoss().to(device) # 손실함수
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate) # SGD, momentum, RMSprop 요소를 합친 내용

In [19]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch
  
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('end')

Epoch: 0001 cost = 5.661354065
Epoch: 0002 cost = 1.619068146
Epoch: 0003 cost = 1.051657677
Epoch: 0004 cost = 0.835085392
Epoch: 0005 cost = 0.714516342
Epoch: 0006 cost = 0.635918915
Epoch: 0007 cost = 0.578721464
Epoch: 0008 cost = 0.535378337
Epoch: 0009 cost = 0.501762033
Epoch: 0010 cost = 0.474781573
Epoch: 0011 cost = 0.452696532
Epoch: 0012 cost = 0.433928639
Epoch: 0013 cost = 0.418381363
Epoch: 0014 cost = 0.404652148
Epoch: 0015 cost = 0.393232107
end


In [20]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8931999802589417
Label:  8
Prediction:  3


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


ReLU

In [21]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(111)
torch.manual_seed(111)
if device == 'cuda':
  torch.cuda.manual_seed_all(111)

print(device)

cpu


In [23]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [26]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transform.ToTensor(),
                        download=True)

In [27]:
print(mnist_train)

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [28]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [29]:
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [30]:
print(linear1)

Linear(in_features=784, out_features=256, bias=True)


In [31]:
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-2.0491,  0.4565, -0.6349,  ...,  1.3137,  0.1490, -0.0198],
        [-0.1050,  0.5532,  1.3073,  ..., -1.3110,  1.4136,  0.4392],
        [ 1.2352,  0.1160, -1.9848,  ..., -1.8623, -0.7066, -1.4861],
        ...,
        [-0.5305, -0.1096, -0.6852,  ..., -1.8234,  0.0563,  1.7481],
        [ 1.0514,  0.4735,  1.1497,  ...,  1.6153,  1.5570, -1.1552],
        [-0.5640,  0.3842, -0.4147,  ...,  1.0944, -0.5508,  0.1636]],
       requires_grad=True)

In [32]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [33]:
criterion = torch.nn.CrossEntropyLoss().to(device)    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 154.986099243
Epoch: 0002 cost = 37.092750549
Epoch: 0003 cost = 23.342712402
Epoch: 0004 cost = 16.339668274
Epoch: 0005 cost = 11.883286476
Epoch: 0006 cost = 8.863075256
Epoch: 0007 cost = 6.599238873
Epoch: 0008 cost = 4.999893188
Epoch: 0009 cost = 3.732627869
Epoch: 0010 cost = 2.817115068
Epoch: 0011 cost = 2.123584032
Epoch: 0012 cost = 1.580967546
Epoch: 0013 cost = 1.307387114
Epoch: 0014 cost = 0.958458960
Epoch: 0015 cost = 0.803140759
Learning finished
